In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Substance_Use_Disorder/GSE161999'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Network preservation reveals shared and unique biological processes associated with chronic alcohol abuse in the NAc and PFC"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['tissue: prefrontal cortex'], 1: ['diagnosis: Alcohol', 'diagnosis: Control'], 2: ['age: 61', 'age: 44', 'age: 62', 'age: 56', 'age: 63', 'age: 42', 'age: 46', 'age: 52', 'age: 43', 'age: 59', 'age: 54', 'age: 39', 'age: 73', 'age: 50', 'age: 51', 'age: 64', 'age: 55', 'age: 47', 'age: 53', 'age: 82', 'age: 57'], 3: ['Sex: Male'], 4: ['rin: 3.6', 'rin: 3.7', 'rin: 3.4', 'rin: 2.1', 'rin: 5.2', 'rin: 5.8', 'rin: 1.4', 'rin: 3.8', 'rin: 2.8', 'rin: 2.9', 'rin: 2.6', 'rin: 2.5', 'rin: 7.8', 'rin: 5', 'rin: 7.2', 'rin: 7.9', 'rin: 4.3', 'rin: 6.6', 'rin: 2.2', 'rin: 8.3', 'rin: 3.1', 'rin: 7.4', 'rin: 4.4', 'rin: 8', 'rin: 3.2'], 5: ['brain weight: 1340', 'brai

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check gene expression data availability
# Assuming from the description provided it is more likely gene expression data
is_gene_available = True

# Determine rows for each variable based on sample characteristics
for key, values in {0: ['tissue: prefrontal cortex'], 1: ['diagnosis: Alcohol', 'diagnosis: Control'], 2: ['age: 61', 'age: 44', 'age: 62', 'age: 56', 'age: 63', 'age: 42', 'age: 46', 'age: 52', 'age: 43', 'age: 59', 'age: 54', 'age: 39', 'age: 73', 'age: 50', 'age: 51', 'age: 64', 'age: 55', 'age: 47', 'age: 53', 'age: 82', 'age: 57'], 3: ['Sex: Male'], 4: ['rin: 3.6', 'rin: 3.7', 'rin: 3.4', 'rin: 2.1', 'rin: 5.2', 'rin: 5.8', 'rin: 1.4', 'rin: 3.8', 'rin: 2.8', 'rin: 2.9', 'rin: 2.6', 'rin: 2.5', 'rin: 7.8', 'rin: 5', 'rin: 7.2', 'rin: 7.9', 'rin: 4.3', 'rin: 6.6', 'rin: 2.2', 'rin: 8.3', 'rin: 3.1', 'rin: 7.4', 'rin: 4.4', 'rin: 8', 'rin: 3.2'], 5: ['brain weight: 1340', 'brain weight: 1220', 'brain weight: 1480', 'brain weight: 1284', 'brain weight: 1570', 'brain weight: 1400', 'brain weight: 1490', 'brain weight: 1510', 'brain weight: 1380', 'brain weight: 1500', 'brain weight: 1520', 'brain weight: 1230', 'brain weight: 1200', 'brain weight: 1360', 'brain weight: 1300', 'brain weight: 1635', 'brain weight: 1616', 'brain weight: 1420', 'brain weight: 1460', 'brain weight: 1370', 'brain weight: 1362', 'brain weight: 1631', 'brain weight: 1534', 'brain weight: 1426', 'brain weight: 1560', 'brain weight: 1390', 'brain weight: 1188'], 6: ['ph: 6.93', 'ph: 6.6', 'ph: 6.56', 'ph: 6.51', 'ph: 6.94', 'ph: 6.5', 'ph: 6.65', 'ph: 6.76', 'ph: 6.78', 'ph: 6.43', 'ph: 6.57', 'ph: 6.52', 'ph: 6.41', 'ph: 6.3', 'ph: 6.53', 'ph: 6.26', 'ph: 6.21', 'ph: 6.59', 'ph: 6.35', 'ph: 7.02', 'ph: 6.39', 'ph: 6.74', 'ph: 6.37', 'ph: 6.89', 'ph: 6.75', 'ph: 6.24', 'ph: 6.84', 'ph: 6.8']}.items():
    if 'diagnosis: Alcohol' in values or 'diagnosis: Control' in values:
        trait_row = key
    if 'age:' in values[0]:
        age_row = key
    if 'Sex:' in values[0]:
        gender_row = key
        
# Define conversion functions
def convert_trait(trait_str):
    if ':' in trait_str:
        trait = trait_str.split(':')[-1].strip().lower()
        if trait == 'alcohol':
            return 1
        elif trait == 'control':
            return 0
    return None

def convert_age(age_str):
    if ':' in age_str:
        try:
            return float(age_str.split(':')[-1].strip())
        except ValueError:
            return None
    return None

def convert_gender(gender_str):
    if ':' in gender_str:
        gender = gender_str.split(':')[-1].strip().lower()
        if gender == 'male':
            return 1
        elif gender == 'female':
            return 0
    return None

# Save cohort information
save_cohort_info('GSE161999', './preprocessed/Substance_Use_Disorder/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Substance_Use_Disorder', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Substance_Use_Disorder/trait_data/GSE161999.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM4929487': [1.0, 61.0, 1.0], 'GSM4929488': [0.0, 44.0, 1.0], 'GSM4929489': [0.0, 62.0, 1.0], 'GSM4929490': [1.0, 56.0, 1.0], 'GSM4929491': [0.0, 63.0, 1.0], 'GSM4929492': [1.0, 42.0, 1.0], 'GSM4929493': [0.0, 46.0, 1.0], 'GSM4929494': [0.0, 56.0, 1.0], 'GSM4929495': [1.0, 52.0, 1.0], 'GSM4929496': [0.0, 43.0, 1.0], 'GSM4929497': [1.0, 59.0, 1.0], 'GSM4929498': [1.0, 56.0, 1.0], 'GSM4929499': [1.0, 54.0, 1.0], 'GSM4929500': [1.0, 46.0, 1.0], 'GSM4929501': [1.0, 39.0, 1.0], 'GSM4929502': [1.0, 73.0, 1.0], 'GSM4929503': [0.0, 56.0, 1.0], 'GSM4929504': [0.0, 50.0, 1.0], 'GSM4929505': [1.0, 63.0, 1.0], 'GSM4929506': [1.0, 50.0, 1.0], 'GSM4929507': [1.0, 50.0, 1.0], 'GSM4929508': [1.0, 51.0, 1.0], 'GSM4929509': [1.0, 64.0, 1.0], 'GSM4929510': [1.0, 55.0, 1.0], 'GSM4929511': [0.0, 55.0, 1.0], 'GSM4929512': [0.0, 47.0, 1.0], 'GSM4929513': [0.0, 50.0, 1.0], 'GSM4929514': [0.0, 55.0, 1.0], 'GSM4929515': [1.0, 53.0, 1.0], 'GSM4929516': [0.0, 82.0, 1.0], 'GSM4929517': [0.0, 64.0, 1.0], 'GSM492

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['hsa-let-7a-2-star_st', 'hsa-let-7a-star_st', 'hsa-let-7a_st',
       'hsa-let-7b-star_st', 'hsa-let-7b_st', 'hsa-let-7c_st',
       'hsa-let-7d-star_st', 'hsa-let-7d_st', 'hsa-let-7e-star_st',
       'hsa-let-7e_st', 'hsa-let-7f-1-star_st', 'hsa-let-7f-2-star_st',
       'hsa-let-7f_st', 'hsa-let-7g-star_st', 'hsa-let-7g_st',
       'hsa-let-7i-star_st', 'hsa-let-7i_st', 'hsa-miR-100-star_st',
       'hsa-miR-100_st', 'hsa-miR-101-star_st'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library.
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library and name the resulting gene expression dataframe "gene_data".
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Substance_Use_Disorder/gene_data/GSE161999.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Substance_Use_Disorder')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE161999', './preprocessed/Substance_Use_Disorder/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Substance_Use_Disorder/GSE161999.csv'
    unbiased_merged_data.to_csv(csv_path)


No gene data in the dataframe
